# COMP90051 Project 2
**Name:** *enter your name here*

**Student ID:** *your id here*

In [131]:
# You can add additional imports here
import numpy as np
from matplotlib import pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict
# Additional imports
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
import time


## 0. Loading the dataset

In [132]:
# do not edit this cell
# load the data files (download from the LMS)
embedded_images = np.load('images.npy')
labels = np.load('labels.npy')

# split into pool & testing
X_pool, X_test, y_pool, y_test = train_test_split(embedded_images, labels, 
                                                  test_size=0.5, random_state=1234, shuffle=True)

# sample a seed set
np.random.seed(1234)
label2id = defaultdict(list)
for i, label in enumerate(y_pool):
    label2id[label].append(i)
seed_set = []
for label, ids in label2id.items():
    #print(label)
    #print(np.random.choice(ids, size=10, replace=False))
    seed_set.extend(np.random.choice(ids, size=10, replace=False))

In [141]:
#X_pool.shape #(9640, 1000)
# for i in y_pool[:14]:
#     print(i)
# myset = set(y_pool)
# print(len(myset)) # 30 unique labels 
#label2id

#Helper Function
def id2label(seed_id,ListLabel):
    seed_set_label = []
    for label, ids in ListLabel.items():
        for seedid in seed_id:
            if seedid in ids:
                seed_set_label.append(label)  
    return seed_set_label

## 1. Applying logistic regression

In [137]:
def train_logistic_regression(X, y, **args):
 
    from numpy import mean
    from numpy import std
    from sklearn.datasets import make_classification
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedStratifiedKFold
    from sklearn.linear_model import LogisticRegression
    # define dataset
   
    # define the multinomial logistic regression model
    log_reg = LogisticRegression(multi_class='multinomial', solver='sag')
    model = log_reg.fit(X, y)



    y_pred = model.predict(X_test)
    print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

    return model


In [138]:
model1 = train_logistic_regression(X_pool[seed_set],seed_set_label)

Accuracy: 0.3070539419087137
C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
def evaluate_logistic_regression_accuracy(Xt, yt, model):
    """
    Apply logistic regression prediction on dataset Xt and evaluate accuracy against yt,
    returing the accuracy results as a scalar.
    Xt: matrix of real values, size m x d
    yt: vector of string labels, size m
    """
    # your code here
    y_pred = model.predict(Xt)
    #score = model.score(Xt,yt)
    #print('Test Accuracy Score', score)
    #score = accuracy_score(yt, y_pred)
    #print("Accuracy: {}".format(accuracy_score(yt, y_pred)))
    # print("Precision: {}".format(precision_score(yt, y_pred)))
    # print("Recall: {}".format(recall_score(yt, y_pred)))
    # evaluate the model and collect the scores
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, Xt, yt, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
    # report the model performance
    return score
    #print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
  

In [ ]:
# your code here for training, evaluating & plotting results

# Traning model 1 with seed_set -> Subest of y_pool.
model1 = train_logistic_regression(X_pool[seed_set],seed_set)

# Traning model 2 with max pool 
#model2 = train_logistic_regression(X_pool,y_pool)

# Evaluation of models trained
#evaluate_logistic_regression_accuracy(X_test,y_test,model1)  


In [ ]:
# Evaluation of models trained
evaluate_logistic_regression_accuracy(X_test,y_test,model1)  

## 2. Active learning framework with Random selection

In [ ]:
def random_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a model (not used)
    returns a vector of scores of length n. Each entry reflects the priority 
    of the corresponding instance. Higher means better.
    """
    # fill in

    # Creating a score array using random numbers of length of X 
    scores = np.random.randint(1,len(X),len(X))
    return scores
    

In [ ]:
def pool_based_active_learning(X_pool, y_pool, seed_ids,
                               train_func, select_func,
                               max_size, batch_size, **args):
    """
    Perform an active learning simulation, which starts by training on a seed set,
    then iteratively applies the selection function to rank instances in the pool,
    selects the top few instances which are included into the training set and the
    process repeats. 
        X_pool: matrix of n x d
        y_pool: vector of string labels, size n
        seed_ids: initial labelled set set, as a list of indices [0..n-1] into pool
        train_func: function which given (X, y, optional args) returns a trained model
        select_func: function which given (X, optional args) returns a sequence of scores
        max_size: stopping condition for active learning, when labelled data reaches given size
        batch_size: number of instances to be labelled in each iteration
        args: optional arguments passed to training and selection function
    returns the sequence of trained models 
    """

    # 1: U = pool of unlabelled instances, {x} 
    U =  np.delete(X_pool,seed_ids, axis=0)
    # 2: L = set of initial labelled instances, {hx,yi} 
    L = [X_pool[seed_ids],seed_ids]   # L = (x,y)  
    # 3: b = number of instances to label in each step 
    b =  batch
    # 4: for t = 1,2,...,T do 
    for i in range(1,45):  # Loop will run from 300 to 3000 Instances
    # 5:  θ(t) = train(L) 
        Theta_model=train_func(L[0],L[1])
        #########################  Accuracy Block #######################################
        model_accuracy = evaluate_logistic_regression_accuracy(X_test,y_test,Theta_model)
        print('Accuracy after query ',i)   
        print('Mean Accuracy: %.3f (%.3f)' % (mean(model_accuracy), std(model_accuracy)))
        #################################################################################
    # 6:  score all instances in pool, r = select(U) 
        r = select_func(U,Theta_model)
    # 7:  for all j ∈ argmax(b,r) do 
        max_indices = np.argpartition(r,-b)[-b:]
        for j in max_indices:
    # 8:      reveal label y     
    # 9:      add hxj,yji to L
            L[0] = np.append(L[0],[X_pool[j]], axis=0)
            L[1].append(j)
            #print("seed set ",len(seed_set))
            #print("seed id",len(seed_ids))
    # 10:     remove xj from U 
            U = np.delete(X_pool,L[1],axis=0)
             #  Calculate and report our model's accuracy.                       
    # 11:    end for 
    # 12: end for 
    # 13: return {θ(t)}T t=1
    return Theta_model
    ########################################## Not Part of Algo ########################
    
        
     #  Calculate and report our model's accuracy.
        #model_accuracy = evaluate_logistic_regression_accuracy(X_test,y_test,Theta_model)
        #print('Accuracy after query {n}: {acc:0.4f}'.format(n=i + 1, acc=model_accuracy))
    # # Save our model's performance for plotting.
        #performance_history.append(model_accuracy)

    ########################################## Not Part of Algo ########################


In [ ]:
len(seed_set)

In [ ]:
batch = 60
max_size = 3000
models_random = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                    train_logistic_regression, random_select, 
                                    max_size, batch)

In [ ]:
## your code here for evaluation of accuracy and plotting of results

# NOTE: Make Graphs and Check Evaluation Scores

## 3. Uncertainty sampling

In [ ]:
def logistic_regression_entropy_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a discriminative model 
    P(y|x), returns a vector of n entropy values.
    """
    # fill in
    print("X Shape",X.shape)
    from scipy.stats import entropy
    #prob = model.predict_proba(X)   # paper has log 
    prob = model.predict_proba(X)
    #ar = prob.to_numpy()
    #print(ar.shape
    ent  = entropy(prob,axis=1,base=10)
    print("Ent shape ",ent.shape)
    print(ent)
    return ent
    #return ent.argmax()
    

In [ ]:
len(seed_set)
print(X_pool.shape)
print(y_pool.shape)

In [ ]:
models_us = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                 train_logistic_regression, 
                                 logistic_regression_entropy_select, 
                                 max_size, batch)

In [ ]:
## your code here for evaluation of accuracy and plotting of results


## 4. Query by committee

In [ ]:
# don't forget to provide function descriptive comments, like those provided in templates above

def query_by_committee_vote_entropy(X, model, **args):
    pass

def query_by_committee_soft_vote_entropy(X, model, **args):
    pass

def query_by_committee_KL(X, model, **args):
    pass

In [ ]:
def train_committee(X, y, **args):
    pass

In [ ]:
## your code here for training, evaluation, and plotting code

## 5. Hierarchical sampling

In [ ]:
## your code